In [ ]:
import pandas as pd
import os
import re
import numpy as np

arousal_wide_csv_path = 'annotations_dynamic/arousal.csv'
valence_wide_csv_path = 'annotations_dynamic/valence.csv'

split_audio_folder = 'DEAM_audio_split_2'
output_csv_path = 'annotations_split_time_based_2.csv'
clip_duration = 2.0

def transform_wide_to_long(df_wide, value_name):
    """Transforma o DataFrame do formato largo para longo."""
    try:
        df_long = pd.melt(df_wide, id_vars=['song_id'], var_name='sample_ms_col', value_name=value_name)

        df_long['timestamp_ms'] = df_long['sample_ms_col'].str.extract(r'(\d+)').astype(int)
        df_long['timestamp_sec'] = df_long['timestamp_ms'] / 1000.0
        df_long = df_long[['song_id', 'timestamp_sec', value_name]]
        return df_long
    except Exception as e:
        print(f"Erro ao transformar DataFrame para {value_name}: {e}")
        print(f"Colunas no DataFrame original: {df_wide.columns.tolist()}")
        raise

try:
    print(f"Carregando Arousal de: {arousal_wide_csv_path}")
    df_arousal_wide = pd.read_csv(arousal_wide_csv_path)
    df_arousal_wide['song_id'] = df_arousal_wide['song_id'].astype(str)
    df_arousal_long = transform_wide_to_long(df_arousal_wide, 'arousal')
    print(df_arousal_long.head())
    print("Arousal transformado para formato longo.")

    print(f"Carregando Valence de: {valence_wide_csv_path}")
    df_valence_wide = pd.read_csv(valence_wide_csv_path)
    df_valence_wide['song_id'] = df_valence_wide['song_id'].astype(str)
    df_valence_long = transform_wide_to_long(df_valence_wide, 'valence')
    print("Valence transformado para formato longo.")

    print("Juntando dataframes de arousal e valence...")
    df_continuous_long = pd.merge(df_arousal_long, df_valence_long, on=['song_id', 'timestamp_sec'], how='inner')
    print(f"Merge completo. Colunas resultantes: {df_continuous_long.columns.tolist()}")
    print(f"Total de anotações combinadas (linhas): {len(df_continuous_long)}")
    print(f"Primeiras linhas combinadas:\n{df_continuous_long.head()}")

except FileNotFoundError as e:
    print(f"ERRO: Arquivo CSV não encontrado: {e}. Verifique os caminhos.")
    exit()
except Exception as e:
    print(f"ERRO durante o carregamento ou transformação inicial: {e}")
    exit()

split_files = [f for f in os.listdir(split_audio_folder) if f.endswith('.wav')]
print(f"\nEncontrados {len(split_files)} arquivos .wav em {split_audio_folder}")

new_annotations_list = []

for filename in split_files:
    match = re.match(r'(\d+)_(\d+)\.wav', filename)
    if not match:
        print(f"Aviso: Não foi possível parsear o nome do arquivo: {filename}. Pulando.")
        continue

    original_song_id_str = match.group(1)
    clip_index = int(match.group(2))
    clip_filename_no_ext = f"{original_song_id_str}_{clip_index}"

    start_time = clip_index * clip_duration
    end_time = (clip_index + 1) * clip_duration

    mask = (
        (df_continuous_long['song_id'] == original_song_id_str) &
        (df_continuous_long['timestamp_sec'] >= start_time) &
        (df_continuous_long['timestamp_sec'] < end_time)
    )
    print(original_song_id_str, start_time, end_time)
    relevant_annotations = df_continuous_long[mask]
    if not relevant_annotations.empty:
        mean_arousal = relevant_annotations['arousal'].mean()
        mean_valence = relevant_annotations['valence'].mean()

        new_annotations_list.append({
            'song_id': clip_filename_no_ext,
            'arousal': mean_arousal,
            'valence': mean_valence
        })
        # print(f"Clipe {clip_filename_no_ext}: Intervalo [{start_time:.1f}s, {end_time:.1f}s) - Arousal={mean_arousal:.4f}, Valence={mean_valence:.4f}")

    else:
        print(f"Aviso: Nenhuma anotação contínua encontrada para o clipe {clip_filename_no_ext} (Intervalo: [{start_time:.1f}s - {end_time:.1f}s)). Clipe será ignorado.")

if new_annotations_list:
    df_new_annotations = pd.DataFrame(new_annotations_list)
    df_new_annotations.to_csv(output_csv_path, index=False)
    print(f"\nNovas anotações salvas em: {output_csv_path}")
    print(f"Total de clipes com anotações válidas: {len(df_new_annotations)}")
    print(f"Primeiras linhas do novo CSV:\n{df_new_annotations.head()}")
else:
    print("\nNenhuma anotação válida foi gerada. Verifique os caminhos e os dados.")

Carregando Arousal de: annotations_dynamic/arousal.csv
  song_id  timestamp_sec   arousal
0       2           15.0 -0.109386
1       3           15.0 -0.110846
2       4           15.0  0.222327
3       5           15.0 -0.255613
4       7           15.0  0.464234
Arousal transformado para formato longo.
Carregando Valence de: annotations_dynamic/valence.csv
Valence transformado para formato longo.
Juntando dataframes de arousal e valence...
Merge completo. Colunas resultantes: ['song_id', 'timestamp_sec', 'arousal', 'valence']
Total de anotações combinadas (linhas): 2203846
Primeiras linhas combinadas:
  song_id  timestamp_sec   arousal   valence
0       2           15.0 -0.109386 -0.073341
1       3           15.0 -0.110846 -0.189702
2       4           15.0  0.222327  0.154286
3       5           15.0 -0.255613  0.149053
4       7           15.0  0.464234  0.342816

Encontrados 46925 arquivos .wav em DEAM_audio_split_2
1705 34.0 36.0
1845 32.0 34.0
1562 40.0 42.0
199 42.0 44.0
813 1